In [1]:
#pip install C:\Users\Aman\Downloads\gmpy2-2.0.8-cp37-cp37m-win_amd64.whl
import gmpy2 as mp
import numpy as np
import math

vigKeyLen = 0
vigKey = ""
blockLen = 0
charsToNum = {}
numToChars = {}
charSpace = 29
ca_signature = "this key has been signed by ca_signature"

In [375]:
def readFile(fname):
    '''Read File from the directory'''
    try:
        text = ""
        with open(fname,'r') as openFile:
            t = openFile.read().replace('\n','')
        openFile.close()
        return t.lower()
    except:
        print("No such file found in directory. Please check")
        return

def writeFile(fname, text):
    '''write to File from the directory'''
    try:
        with open(fname,'w') as openFile:
            openFile.write(text)
        openFile.close()
        return
    except:
        print("No such file found in directory. Please check")
        return

def charsDictGen():  
    '''#form dictionary of chars a-z and map them to 0-25 numbers'''
    charsDict = {"a":None}
    init = 'a'
    for i in range(0,26):
        charsDict[init] = i
        init = chr(ord(init) + 1)
    charsDict[' '] = 26
    charsDict[','] = 27
    charsDict['.'] = 28
    return charsDict

def numbersDictGen():  
    '''#form dictionary of 0-25 numbers and map them to chars a-z '''
    numsDict = {0:None}
    init = 'a'
    for i in range(0,26):
        numsDict[i] = init
        init = chr(ord(init) + 1)
    numsDict[26] = ' '
    numsDict[27] = ','
    numsDict[28] = '.'
    return numsDict

def getMsgBlocks(text, l):
    '''Returns list of message blocks of length l'''
    msgBlocks = []
    tempBlock = ""
    
    for char in text:
        tempBlock += char
        if(len(tempBlock) == l):
            msgBlocks.append(tempBlock)
            tempBlock = ""
    msgBlocks.append(tempBlock) 
    return msgBlocks

def enVigenere(text, key):
    '''Encrypt the given message from vigenere cipher.'''
    
    msgBlocks = getMsgBlocks(text, len(key))
    enMsg = ""
    for block in msgBlocks:
        for i in range(len(block)):
            enMsg += numToChars[(charsToNum[block[i]] + charsToNum[key[i]])%charSpace]
    
    return enMsg

def deVigenere(text, key):
    
    msgBlocks = getMsgBlocks(text, len(key))
    deMsg = ""
    for block in msgBlocks:
        for i in range(len(block)):
            deMsg += numToChars[(charsToNum[block[i]] - charsToNum[key[i]])%charSpace]
    
    return deMsg


def genStrongPrime(bits):
    '''Generates strong prime based on BPSW Strong probable prime test'''
    notStrongPrime = True
    prp = mp.next_prime(2**bits)
    count = 0
    while notStrongPrime:
        if mp.is_strong_bpsw_prp(prp):
            notStrongPrime = False
        elif not mp.is_strong_bpsw_prp(prp):
            prp = mp.next_prime(prp)
    return prp


def getPQ(b):
    '''returns p and q for rsa for generating n = p*q of min length "b" bits'''
    b = round(b/2) + int(np.random.choice([0,1,2,3,4,5,6,7,8,9], 1)[0])
    p = genStrongPrime(b)
    
    b += int(np.random.choice([10,11,12,13,14,15,16,17,18,19], 1)[0])
    q = genStrongPrime(b)
    return p,q


def rsaKeyGen(bits):
    '''RSA Private and Public Key Generator'''
    p, q = getPQ(bits)
    n = mp.mul(p,q)
    phi = mp.mul(p-1, q-1)
    e = 2**445 + 1
    notCoprime = True
    while notCoprime:
        if mp.gcd(e, phi) == 1:
            notCoprime = False
        else:
            e += 2
    
    d = mp.invert(e, phi)
    #print("phi\n", phi)
    #print(mp.t_mod(mp.mul(e,d), phi) == 1)
    return e, n, d

def signDigitally(text):
    '''Sign a Document Digitally by CA by his private key'''
    pk = readFile("public_directory/public_" + "ca" + ".txt").split()
    sk = readFile("secret_" + "ca" + ".txt").split()
    
    n = mp.mpz(pk[1])
    d = mp.mpz(sk[0])
    blockLenDsc = 40
    print("Block Length", blockLenDsc, type(blockLenDsc))
                
    msg = 0
    for i in range(blockLenDsc):
        msg += mp.mul(charsToNum[text[i]], mp.mpz(charSpace)**(blockLenDsc -1 -i))     

    eMsg = mp.powmod(msg, d, n) #sign by CA Secret Key
    
    return eMsg

def deCaSignature(text):
    '''recover sign of CA by his public key'''
    pk = readFile("public_directory/public_" + "ca" + ".txt").split()
    
    e = int(pk[0])
    n = mp.mpz(pk[1])
    blockLenDsc = 40
    
    dSign = mp.powmod(text, e, n) #Check signature Validity by CA Pbblic key

    remainder = dSign
    m = []
    decryptedSign = ""
    for i in range(blockLenDsc):
        quot, remainder = mp.t_divmod(remainder, mp.mpz(charSpace)**(blockLenDsc -1 -i))
        m.append(numToChars[mp.t_mod(quot, charSpace)])
    decryptedSign += ''.join(m)
        
    return decryptedSign

def publishCaSignedKeys():

    e_ca, n_ca, d_ca = rsaKeyGen(512)
    ca_signature = "this key has been signed by ca signature"
    dsc = signDigitally(ca_signature)
    print(dsc)
    
    writeFile("public_directory/public_ca.txt", str(e_ca) + " " + str(n_ca) + " " + str(dsc))
    writeFile("secret_ca.txt",str(d_ca) + " " + str(n_ca) + " " + str(dsc))

    e_a, n_a, d_a = rsaKeyGen(1024)
    writeFile("public_directory/public_a.txt", str(e_a) + " " + str(n_a) + " " + str(dsc))
    writeFile("secret_a.txt",str(d_a) + " " + str(n_a) + " " + str(dsc))

    e_b, n_b, d_b = rsaKeyGen(1024)
    writeFile("public_directory/public_b.txt", str(e_b) + " " + str(n_b) + " " + str(dsc))
    writeFile("secret_b.txt",str(d_b) + " " + str(n_b) + " " + str(dsc))
    
    return

def prepareMsgForRSA(msgEnVig):
    return numToChars[vigKeyLen] + vigKey + msgEnVig


def sendSecureMessage(msg, receiver, sender):
    MsgEn_Vig = enVigenere(msg, vigKey)
    
    MsgForRSA = prepareMsgForRSA(MsgEn_Vig)
    print("MSG FOR RSA", MsgForRSA)
    
    MsgEn_SenderPvtKey = enRSA(MsgForRSA, sender, receiver, "d") #d stands for Decrypt Operation with secret key
    print("MsgEn_SenderPvtKey", MsgEn_SenderPvtKey)
    
    MsgEn_ReceiverPubKey = enRSA(MsgEn_SenderPvtKey, sender, receiver, "e") #e stands for Encrypt Operation with public key
    print("MsgEn_ReceiverPubKey", MsgEn_ReceiverPubKey)
    
    writeFile("sent_message.txt", MsgEn_ReceiverPubKey)


def calcBlockSize(n):
    a = mp.mpz(n)
    match = False
    r = 0
    while mp.mpz(charSpace)**r < a:
        r += 1
    return r

def enRSA(text, sender, receiver, operation):
    '''RSA Encryption with public key of receiver'''
    
    operator = 0
    
    if(operation == "d"):
        sk = readFile("secret_" + sender + ".txt").split()
        d = mp.mpz(sk[0].strip())
        n = mp.mpz(sk[1].strip())
        N = n
        operator = d
    elif(operation == "e"):
        pk = readFile("public_directory/public_" + receiver + ".txt").split()
        e = int(pk[0].strip())
        n = mp.mpz(pk[1].strip())
        N = n
        operator = e
    else:
        print("Please specify the operation as x = e / d")
        return
    print("\n N Bits", mp.bit_length(N))    
    blockLen = calcBlockSize(N)
    print("Block Length", blockLen, type(blockLen))
    msgBlocks = getMsgBlocks(text, blockLen)
    if(len(msgBlocks[-1]) != blockLen):
        for i in range(blockLen - len(msgBlocks[-1])):
            msgBlocks[-1] += numToChars[np.random.randint(0, 2)]
            
    encryptedMsg = []
    for block in msgBlocks:
        msg = 0
        for i in range(blockLen):
            msg += mp.mul(charsToNum[block[i]], mp.mpz(charSpace)**(blockLen -1 -i))   
        print("\n Orig Msg Bits", mp.bit_length(msg))
        print("\n Orig Msg\n", msg)
        print("\nOperator e\n", operator)
        eMsg = mp.powmod(msg, operator, N)
        encryptedMsg.append(eMsg)
    
    print("\nencryptedMsgBlock", encryptedMsg)
    eMsg_Eng = ""
      
    enMsg = ""
    for msg in encryptedMsg:
        #qdash, remainder = mp.t_divmod(msg, mp.mpz(charSpace)**(blockLen))
        #print("\nQDash", qdash)
        remainder = msg
        m = []
        for i in range(blockLen):
            quot, remainder = mp.t_divmod(remainder, mp.mpz(charSpace)**(blockLen -1 -i))
            m.append(numToChars[quot])
        enMsg += ''.join(m)
        print("EMsg\n", ''.join(m))
    
    return enMsg
        
def deRSA(text, sender, receiver, operation):
    '''RSA Decryption with public key of receiver'''    
    
    operator = 0
    
    if(operation == "d"):
        sk = readFile("secret_" + receiver + ".txt").split()
        d = mp.mpz(sk[0].strip())
        n = mp.mpz(sk[1].strip())
        operator = d
        N = n
    elif(operation == "e"):
        pk = readFile("public_directory/public_" + sender + ".txt").split()
        e = int(pk[0].strip())
        n = mp.mpz(pk[1].strip())
        operator = e
        N = n
    else:
        print("Please specify the operation as x = e / d")
        return
    blockLen = calcBlockSize(N)
    msgBlocks = getMsgBlocks(text, blockLen)
    print("\nMSG BLOCKS DE RECEIVED\n", msgBlocks[0])
    
    decryptedMsg = []
    for block in msgBlocks:
        msg = 0
        if len(block) != 0:
            for i in range(blockLen):
                msg += mp.mul(charsToNum[block[i]], mp.mpz(charSpace)**(blockLen -1 -i))
            
            #msg += mp.mpz(charSpace)**(blockLen-1)
        
            decryptedMsg.append(msg)
    deMsgBlocks = []  
    tempDeMsg = ""
    
    
    for block in decryptedMsg:
        print("\nBroken Blocks:\n" ,block)
        print("\n nBroken Msg Bits", mp.bit_length(block))
        print("\nOperator d\n", operator)
        print("\n N: \n", N )
        remain = mp.powmod(mp.mpz(block), operator, N)
        deMsgBlocks.append(remain)
        print("After Decryptoin\n", remain)
      
    decryptedMsg = ""
    for msg in deMsgBlocks:
        remainder = msg
        m = []
        for i in range(blockLen):
            quot, remainder = mp.t_divmod(remainder, mp.mpz(charSpace)**(blockLen -1 -i))
            m.append(numToChars[mp.t_mod(quot, charSpace)])
        decryptedMsg += ''.join(m)
        
    return decryptedMsg


In [381]:
#publishCaSignedKeys()
msg = readFile("message.txt")
msg_rsa_de = enRSA(msg, "a", "b", "e")
#print("msg_rsa_encrypted", msg_rsa_encrypted)

msg_rsa_de = deRSA(msg_rsa_de, "a", "b", "d")
print("msg_rsa_decrypted:\n", msg_rsa_de)


 N Bits 1045
Block Length 215 <class 'int'>

 Orig Msg Bits 1041

 Orig Msg
 21364215110687517830671181658901091944170463484751146056165531394486585877438010334806999129548289583468004482747233531286024240635581712141806692595774153064095782581929098846147925351660021329726210248995136061947610727098592505086191623869515524898255821853550426336126060046692189069926284119049580549315855425

Operator e
 90854840536950861318665475986000566794205170085914757535186274897579911014174740415773881339220445695095315200783272241691825203576835

 Orig Msg Bits 1043

 Orig Msg
 62135457480004719072260555131960327488917725099538013304652385693139703890804899536260233649466293269149047616730504290488955805764460477950982284076313783527828948935769925883680022740498995539183179535706843457295944074892583331115232582764503924604559501309418271885178132559297317483158603573596567312992808654

Operator e
 90854840536950861318665475986000566794205170085914757535186274897579911014174740415773881339220445

In [303]:
#print("MSG", mp.bit_length(17692020009273691603376199479524620097808215333324087942024941283891017083918341985835402673331391793132973809690698367238231176973674770059238312752465556972052))
#print("n", mp.bit_length(14059105607947488696282932836518693308967803494693489478439861164411992439600615480631803604099857686570861644171994571584855813806269673001321217634624497258999))


n = mp.mpz(14059105607947488696282932836518693308967803494693489478439861164411992439600615480631803604099857686570861644171994571584855813806269673001321217634624497258999)
r = mp.mpz(charSpace)**110
print(n>r)

False


In [199]:
msg = mp.mpz(135413922060639231978798669016428840332583244210772138588236533813686468279486578005159894648663919376016349538362441920670349085867172402517979)
#print("msg:\n", msg)
e = 65537 
phi = 121416805764108066932466369176469931665440380684482584059201563258175697791234343016 
n = mp.mpz(121416805764108066932466369176469931665451541946117590840803161945299459788570099847)

d = mp.mpz(11364125403314751706116372560972680483327147948771170035539350123222755546507033585 )

msg = mp.mpz(121416805764108066932466369176469931665451541946117590840803161945299459788570099848)
print("Bits size msg", mp.bit_length(msg))
print("Bits size e", mp.bit_length(e))
print("Bits size d", mp.bit_length(d))
print("Bits size n", mp.bit_length(n))
b = mp.powmod(msg, e, n)
#print("b\n", b)

print("demsg\n", mp.powmod(b, d, n))



#block = mp.mpz(190459789025112130935632505941732185815976614667592029637716294576394326453151808007601876414765668581575926172639875314151127670734408185306216379794486665938)
#c = mp.powmod(block, d, n)
#print(c)

Bits size msg 277
Bits size e 17
Bits size d 273
Bits size n 277
demsg
 1


In [102]:
'''Initialize Variables'''
charsToNum = charsDictGen() #Generate Characters to Number Dictionary
numToChars = numbersDictGen() #Generate Numbers to Characters Dictionary
vigKey = readFile("vigenere_key.txt") #read vigenere cipher key 
vigKeyLen = len(vigKey)

'''Publish CA Signed Public Keys in Public Directory and Send Secret Keys to respective users'''
publishCaSignedKeys() #Publish Private and Public Keys signed by CA

'''Take input from User'''
Message = readFile("message.txt") #Read Msg from message text file
SendTo = str(input("SEND MESSAGE TO: (a or b)")) #Input Receiver's Name
From = str(input("FROM: (a or b)")) #Input Sender's Name

'''Send Secure Message'''
sendSecureMessage(Message, SendTo, From) #Send Message With High Security


Block Length 40 <class 'int'>
6324439457851661129760239086434485213756694227388653109809333890973326153185558342672860969288840992229301737959252999031471897369664493862183081957048241190132855
SEND MESSAGE TO: (a or b)g
FROM: (a or b)nh
MSG FOR RSA xwhatthefuckiamdoinghereysa,hlbp.nv  cksfbg bsymeidq ljrr,wcqvdgkxh.oeyjiwxuxfcnffkzrqscdvzvvtvnxqvjckjoqrjoou.yfbsykuidzeecnqylezaeio
No such file found in directory. Please check


AttributeError: 'NoneType' object has no attribute 'split'

In [178]:

dsc = signDigitally("this key has been signed by ca signature")
print(dsc)

caSig = deCaSignature(dsc)
print(caSig)

Block Length 40 <class 'int'>
75742330106527715893802228003471513480665784825658476368699466049672815757156429760
this key has been signed by ca signature


In [14]:

msg = readFile("message.txt")
msg_rsa_de = enRSA(msg, "a", "a", "d")
#print("msg_rsa_encrypted", msg_rsa_encrypted)

msg_rsa_de = deRSA(msg_rsa_de, "a", "a", "e")
print("msg_rsa_decrypted:\n", msg_rsa_de)

Block Length 35 <class 'int'>
encryptedMsgBlock [mpz(150628460692382029960935376048261068618647462280468650218816260891686833263503782736930930107073582388979540293979448050658689196272015453677364391716687548629079), mpz(98036692962781250897284094813185541094438015140111789863912550686393249104278150098311647616811189291926447052579647948214211233747994147951569766603785096450793), mpz(62068315462920894975706624900684980009759171172640394454719866333155641367459155729881763752167439260424591266278530055117923386035765804918263338549531479277192), mpz(172099684651367667334752321380681698759548215161012341796640652369501270573813983439695525507856878198731463633800336167415270251852048394937819353936285636879408), mpz(187271446737905020275682433936346122562139490416388067795409464494595123986890782878673627481991083993210595994322072153424405951154203508408378940579694072151833), mpz(27139352335370367209358061981196843252162101837579545414815334835795943116842441708172126948046880101528

KeyError: mpz(2569326874044173103127153168723737263603943455362)

In [293]:
x = mp.mpz(216567016859265086196421767469427105974602390337583470969872845751045696229323397017266821541323651326589641320747251056020147115829366508099343748448163302075747)
y = mp.mpz(845877578333521112962751409095693043306229346037993)
z = mp.mpz(29)**35
print(mp.t_mod(x, y))

223136545594579814878750349981895839857554000360047


In [33]:
print(mp.t_mod(mp.mpz(7108)**3, 28471))
print(mp.t_mod(mp.mpz(27989)**1867, 28471))

27989
7108


In [27]:
print(mp.invert(mp.mpz(3), mp.mpz(2800)))

1867


In [32]:
print(mp.t_divmod(mp.mpz(27989), 29**3))
print(mp.t_divmod(mp.mpz(3600), 29**2))
print(mp.t_divmod(mp.mpz(236), 29**1))

(mpz(1), mpz(3600))
(mpz(4), mpz(236))
(mpz(8), mpz(4))


In [47]:
a = mp.mpz(106611892003818054998249087386483466645805390339475749725598387269677608194530835642660852703736114770005864561533566872805736602447315775896509439928483225513363)
b = mp.mpz(3091744868110723594949223534208020532728356319844796742042353230820650637641394233637164728408347328330170072284473439311366361470972157500998773757926013539887527)



print(r)


112


In [248]:
s = "abcxs "
print(s.strip())

abcxs
